# Day 8: Resonant Collinearity

You find yourselves on the [roof](https://adventofcode.com/2016/day/25) of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar _huge antenna_. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific _frequency_ indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific _antinodes_ based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency `a`, they create the two antinodes marked with `#`:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; `A` and `a` count as different frequencies. However, antinodes _can_ occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital `A` creates no antinodes but has a lowercase-`a`\-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost `A`\-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost `A`\-frequency antenna overlaps with a `0`\-frequency antinode, there are `_14_` total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. _How many unique locations within the bounds of the map contain an antinode?_

In [ ]:
$sample_data = (@'
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
'@).Replace("`r", '') -split "`n" 

$input_data = Get-Content .\data.txt

In [9]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )
    if($sample) {
        $data = $sample_data
        $out = "With *SAMPLE* dataset, part "
    } else {
        $data = $input_data
        $out = "Part "
    }
    $out += $part2 ? '2: ' : '1: '

    # Get bounds of map
    $mapWidth = $data[0].length
    $mapHeight = $data.count

    # Get the coords of every antenna
    $antennae = [System.Collections.ArrayList]::new()
    for($y=0; $y -lt $mapHeight; $y++) {
        for($x=0; $x -lt $mapWidth; $x++) {
            if($data[$y][$x] -eq '.') {
                continue
            }
            $antennae.add([pscustomobject]@{
                frequency = [char]$data[$y][$x]
                y = $y
                x = $x
            }) | Out-Null
        }
    }


    Write-Debug "Found $($antennae.Count) antennae."
    $antinodes = [System.Collections.ArrayList]::new()
    $antennae = $antennae | Sort-Object -Property frequency
    for($i=0; $i -lt $antennae.count; $i++){
        Write-Debug "Checking antinodes of frequency $($antennae[$i].frequency) at [$($antennae[$i].y),$($antennae[$i].x)]"
        $j=$i+1
        while($antennae[$j].frequency -eq $antennae[$i].frequency) {
            Write-Verbose "Checking against antenna at [$($antennae[$j].y),$($antennae[$j].x)]"
            $xOffset = $antennae[$i].x - $antennae[$j].x # 3
            $yOffset = $antennae[$i].y - $antennae[$j].y # -1
            # check first antinode
            $yAntinode = $antennae[$i].y + $yOffset
            $xAntinode = $antennae[$i].x + $xOffset 
            if($yAntinode -lt $mapWidth -and $yAntinode -ge 0 -and
                    $xAntinode -lt $mapHeight -and $xAntinode -ge 0 ) {
                Write-Debug "Antinode A at [$yAntinode,$xAntinode] is within the bounds of the map!"
                $antinodes.add([pscustomobject]@{
                    frequency = $antennae[$i].frequency
                    y = $yAntinode
                    x = $xAntinode
                }) | Out-Null
            }
            # check second antinode
            $yAntinode = $antennae[$j].y - $yOffset
            $xAntinode = $antennae[$j].x - $xOffset 
            if($yAntinode -lt $mapWidth -and $yAntinode -ge 0 -and
                    $xAntinode -lt $mapHeight -and $xAntinode -ge 0 ) {
                Write-Debug "Antinode B at [$yAntinode,$xAntinode] is within the bounds of the map!"
                $antinodes.add([pscustomobject]@{
                    frequency = $antennae[$i].frequency
                    y = $yAntinode
                    x = $xAntinode
                }) | Out-Null
            }
            $j++
        }
    }

    $uniqueAntinodes = ($antinodes | Sort-Object -Unique -Property y,x).count

    $out += "found a total of $($antinodes.count) antinodes, $uniqueAntinodes in unique locations."
    Write-Output $out
}

dostuff -sample -Debug
dostuff

DEBUG: Found 7 antennae.
DEBUG: Checking antinodes of frequency 0 at [1,8]
DEBUG: Antinode A at [0,11] is within the bounds of the map!
DEBUG: Antinode B at [3,2] is within the bounds of the map!
DEBUG: Antinode B at [5,6] is within the bounds of the map!
DEBUG: Antinode B at [7,0] is within the bounds of the map!
DEBUG: Checking antinodes of frequency 0 at [2,5]
DEBUG: Antinode A at [1,3] is within the bounds of the map!
DEBUG: Antinode B at [4,9] is within the bounds of the map!
DEBUG: Antinode A at [0,6] is within the bounds of the map!
DEBUG: Antinode B at [6,3] is within the bounds of the map!
DEBUG: Checking antinodes of frequency 0 at [3,7]
DEBUG: Antinode A at [2,10] is within the bounds of the map!
DEBUG: Antinode B at [5,1] is within the bounds of the map!
DEBUG: Checking antinodes of frequency 0 at [4,4]
DEBUG: Checking antinodes of frequency A at [5,6]
DEBUG: Antinode A at [2,4] is within the bounds of the map!
DEBUG: Antinode B at [11,10] is within the bounds of the map!
D

# Part Two

Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at _any grid position_ exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three `T`\-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three `T`\-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to `_9_`.

The original example now has `_34_` antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. _How many unique locations within the bounds of the map contain an antinode?_

In [14]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )
    if($sample) {
        $data = $sample_data
        $out = "With *SAMPLE* dataset, part "
    } else {
        $data = $input_data
        $out = "Part "
    }
    $out += $part2 ? '2: ' : '1: '

    # Get bounds of map
    $mapWidth = $data[0].length
    $mapHeight = $data.count

    # Get the coords of every antenna
    $antennae = [System.Collections.ArrayList]::new()
    for($y=0; $y -lt $mapHeight; $y++) {
        for($x=0; $x -lt $mapWidth; $x++) {
            if($data[$y][$x] -eq '.') {
                continue
            }
            $antennae.add([pscustomobject]@{
                frequency = [char]$data[$y][$x]
                y = $y
                x = $x
            }) | Out-Null
        }
    }


    Write-Debug "Found $($antennae.Count) antennae."
    $antinodes = [System.Collections.ArrayList]::new()
    $antennae = $antennae | Sort-Object -Property frequency
    for($i=0; $i -lt $antennae.count; $i++){
        Write-Debug "Checking antinodes of frequency $($antennae[$i].frequency) at [$($antennae[$i].y),$($antennae[$i].x)]"
        $j=$i+1
        while($antennae[$j].frequency -eq $antennae[$i].frequency) {
            Write-Verbose "Checking against antenna at [$($antennae[$j].y),$($antennae[$j].x)]"
            $xOffset = $antennae[$i].x - $antennae[$j].x
            $yOffset = $antennae[$i].y - $antennae[$j].y 
            $antinodePropogationLevel = 1
            if($part2) {
                $antinodePropogationLevel = 100
            }
            $antinodeAinbounds = $true
            $antinodeBinbounds = $true
            for($k=1; $k -lt $antinodePropogationLevel; $k++) {
                $yAntinode = $antennae[$i].y + $yOffset * $k
                $xAntinode = $antennae[$i].x + $xOffset * $k
                if($yAntinode -lt $mapWidth -and $yAntinode -ge 0 -and
                        $xAntinode -lt $mapHeight -and $xAntinode -ge 0 ) {
                    Write-Debug "Antinode A at [$yAntinode,$xAntinode] is within the bounds of the map!"
                    $antinodes.add([pscustomobject]@{
                        frequency = $antennae[$i].frequency
                        y = $yAntinode
                        x = $xAntinode
                    }) | Out-Null
                } else {
                    $antinodeAinbounds = $false
                }
                # check second antinode
                $yAntinode = $antennae[$j].y - $yOffset * $k
                $xAntinode = $antennae[$j].x - $xOffset * $k
                if($yAntinode -lt $mapWidth -and $yAntinode -ge 0 -and
                        $xAntinode -lt $mapHeight -and $xAntinode -ge 0 ) {
                    Write-Debug "Antinode B at [$yAntinode,$xAntinode] is within the bounds of the map!"
                    $antinodes.add([pscustomobject]@{
                        frequency = $antennae[$i].frequency
                        y = $yAntinode
                        x = $xAntinode
                    }) | Out-Null
                } else {
                    $antinodeBinbounds = $false
                }
                if(!$antinodeAinbounds -and !$antinodeBinbounds) {
                    Write-Debug "After $k rounds, neither antinode is in bounds."
                    break
                }
            }
            if($antinodeAinbounds -or $antinodeBinbounds) {
                Write-Warning "Antinode propagation level was likely exceeded."
            }
            $j++
        }
    }

    $uniqueAntinodes = ($antinodes + $antennae | Sort-Object -Unique -Property y,x).count

    $out += "found a total of $($antinodes.count) antinodes and $($antennae.count) antennae; $uniqueAntinodes in unique locations."
    Write-Output $out
}

dostuff -sample -Debug -part2
dostuff -part2

DEBUG: Found 7 antennae.
DEBUG: Checking antinodes of frequency 0 at [1,8]
DEBUG: Antinode A at [0,11] is within the bounds of the map!
DEBUG: Antinode B at [3,2] is within the bounds of the map!
DEBUG: After 2 rounds, neither antinode is in bounds.
DEBUG: Antinode B at [5,6] is within the bounds of the map!
DEBUG: Antinode B at [7,5] is within the bounds of the map!
DEBUG: Antinode B at [9,4] is within the bounds of the map!
DEBUG: Antinode B at [11,3] is within the bounds of the map!
DEBUG: After 5 rounds, neither antinode is in bounds.
DEBUG: Antinode B at [7,0] is within the bounds of the map!
DEBUG: After 2 rounds, neither antinode is in bounds.
DEBUG: Checking antinodes of frequency 0 at [2,5]
DEBUG: Antinode A at [1,3] is within the bounds of the map!
DEBUG: Antinode B at [4,9] is within the bounds of the map!
DEBUG: Antinode A at [0,1] is within the bounds of the map!
DEBUG: Antinode B at [5,11] is within the bounds of the map!
DEBUG: After 3 rounds, neither antinode is in boun